# Beginning of the Official Jupyter Notebook for the Sports Bets Analyzer

In [ ]:
# The first thing I'd like to do is setup the API Data Collection

In [ ]:
# Next I would like to setup the prop analyzer program so expected output based on given data by the API

In [ ]:
# After that we will need a configuration file to configure the collection of the API Data to what we need for the SBA

In [ ]:
# Second to last I want to create a program for choosing the best prop combinations

In [ ]:
# Lastly, we should have our main program 